In [41]:
import sys
import numpy as np
import pandas as pd
sys.path.append('D:/research/jokesclustering/') # ваш путь до корня проекта
from vector_clustering.data.manager import get_jokes_as_dataframe, read_lines_from_file
from gensim.utils import simple_preprocess
from gensim import models, corpora

In [7]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
from pymorphy2 import MorphAnalyzer
analyzer = MorphAnalyzer()

C:\Users\PC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
jokes_frame = get_jokes_as_dataframe()
jokes_frame.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Одна белка случайно попробовала пиво и поняла,..."
4,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...


### Функции предобработки текста
лемматизаторы, чистка стоп-слов, любой необходимый препроцессинг

In [3]:
def gensim_simple_preprocess(text, min_len=3, max_len=1000000):
    ## приводит к нижнему регистру, удаляет знаки препинания, оставляет слова между (min_length, max_length)
    return ' '.join(simple_preprocess(text, min_len=min_len, max_len=max_len))

In [4]:
def rnnmorph_lemmatizer(text):
    ## альтернатива для pymorphy2: приводит слова к нормальной форме.
    return ' '.join([x.normal_form for x in predictor.predict(text.split())])

def pymorphy_lemmatizer(text):
    return ' '.join(analyzer.parse(x)[0].normal_form for x in text.split())

In [5]:
stop_words = set([x.split('\n')[0] for x in read_lines_from_file('ru_stop_words.txt')])

def stop_words_remove(text):
    ## удаление из текста стоп-слов на основании списка стоп-слов
    return ' '.join([x for x in text.lower().split() if x not in stop_words])

In [85]:
### сделать в качестве словаря только Tfidf ?
## если сделать Tfidf, то можно убрать из словаря самые редкие слова и самые частые
### доделать !?

In [8]:
# определяет порядок вызова функций
# rnnmorph_lemmatizer слишком долгий
cleaners = [gensim_simple_preprocess, stop_words_remove, pymorphy_lemmatizer]

def apply_to_text(t):
    for func in cleaners:
        t = func(t)
    return t

def apply_func_to_texts(texts, func):
    return [func(t) for t in texts]

def apply_to_texts(ts):
    return [apply_to_text(t) for t in ts]

def apply_to_texts_func_by_func(texts):
    for func in cleaners:
        texts = apply_func_to_texts(texts, func)
        print('Applied {}'.format(func))
    return texts

In [9]:
%%time
preprocessed_collection = apply_to_texts_func_by_func(jokes_frame.joke_text.values)

Applied <function gensim_simple_preprocess at 0x0000019B31CEB2F0>
Applied <function stop_words_remove at 0x0000019B29AE1048>
Applied <function pymorphy_lemmatizer at 0x0000019B31CEB598>
Wall time: 3min 28s


### Построение lda модели

In [82]:
number_topics = 100

In [15]:
tokenized_collection = [x.split() for x in preprocessed_collection]
dictionary = corpora.Dictionary(tokenized_collection) # словарь вида: dictionary[word] : numeric_index
corpus = [dictionary.doc2bow(text) for text in tokenized_collection] # каждый документ теперь выглядит так:
# [(12, 3), (25, 2), ... ] - то есть в шутке слово с индексом 12 встретилось 3 раза, слово с индексом 25 - 2 раза и так далее

In [95]:
lda = models.LdaModel(corpus=corpus, 
                      num_topics=number_topics, 
                      id2word=dictionary, 
                      dtype=np.float64,
                      random_state=42)

In [96]:
for idx in range(number_topics):
    print('Topic: {}, words: {}'.format(idx, lda.print_topic(idx, 5)))
    print()

Topic: 0, words: 0.125*"даже" + 0.043*"буква" + 0.028*"немец" + 0.027*"шеф" + 0.023*"ругаться"

Topic: 1, words: 0.235*"или" + 0.167*"конечно" + 0.031*"разница" + 0.022*"собирать" + 0.019*"надеяться"

Topic: 2, words: 0.114*"рабин" + 0.066*"добрый" + 0.053*"бывать" + 0.031*"официант" + 0.027*"камера"

Topic: 3, words: 0.104*"найти" + 0.074*"час" + 0.071*"ехать" + 0.035*"прямо" + 0.034*"прийтись"

Topic: 4, words: 0.172*"теперь" + 0.106*"нибыть" + 0.062*"неделя" + 0.059*"собака" + 0.042*"быть"

Topic: 5, words: 0.109*"подумать" + 0.070*"главный" + 0.068*"выйти" + 0.048*"гаишник" + 0.046*"оказаться"

Topic: 6, words: 0.190*"делать" + 0.135*"ничто" + 0.047*"плохой" + 0.046*"уметь" + 0.040*"попасть"

Topic: 7, words: 0.239*"жена" + 0.155*"дом" + 0.065*"написать" + 0.048*"дочь" + 0.030*"садиться"

Topic: 8, words: 0.209*"тогда" + 0.069*"хоть" + 0.049*"жопа" + 0.031*"общий" + 0.029*"брат"

Topic: 9, words: 0.094*"посмотреть" + 0.089*"должный" + 0.088*"быть" + 0.076*"дед" + 0.049*"стол"

Topi

Topic: 80, words: 0.168*"работать" + 0.095*"москва" + 0.056*"город" + 0.046*"верить" + 0.046*"сэр"

Topic: 81, words: 0.153*"надо" + 0.108*"хорошо" + 0.096*"жить" + 0.065*"маленький" + 0.049*"иметь"

Topic: 82, words: 0.167*"приходить" + 0.083*"значит" + 0.074*"еврей" + 0.044*"семья" + 0.036*"если"

Topic: 83, words: 0.110*"понять" + 0.099*"иван" + 0.050*"война" + 0.033*"почти" + 0.029*"грудь"

Topic: 84, words: 0.188*"знать" + 0.120*"доктор" + 0.056*"молодая" + 0.048*"последний" + 0.028*"фильм"

Topic: 85, words: 0.166*"ребёнок" + 0.070*"сразу" + 0.041*"слушай" + 0.035*"дочка" + 0.034*"родитель"

Topic: 86, words: 0.111*"стоить" + 0.087*"встречаться" + 0.068*"новость" + 0.056*"конец" + 0.048*"давно"

Topic: 87, words: 0.064*"вернуться" + 0.042*"отдать" + 0.032*"лишь" + 0.027*"привести" + 0.025*"метр"

Topic: 88, words: 0.084*"кричать" + 0.068*"квартира" + 0.061*"кот" + 0.057*"миллион" + 0.054*"люся"

Topic: 89, words: 0.116*"отвечать" + 0.064*"нельзя" + 0.041*"можно" + 0.038*"абрам" +

### Присвоение тем шуткам
на выходе получим дата фрейм из одной колонки, эта колонка - самая вероятная тема для данной шутки

In [97]:
most_relevant_topics = [sorted(lda.get_document_topics(bow), key=lambda x: x[1], reverse=True)[0][0] for bow in corpus]

In [98]:
pd.DataFrame({'topic_id' : most_relevant_topics}).to_csv('lda_100_topics.csv')

In [99]:
jokes_frame['topic_id'] = most_relevant_topics

In [100]:
jokes_frame.head()

,joke_text,topic_id
0,"Мужчину трудно задеть за живое, но уж если зад...",43
1,В нашем кемпинге строго запрещено людям разног...,46
2,А как хорошо у девушек начинается: любимый: ми...,37
3,"Одна белка случайно попробовала пиво и поняла,...",48
4,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...,17


### Функции для отображения результатов
теперь тема описывается словами. Чтобы понять, что это за тема и оценить ее качество нужно посмотреть на топ слов и на шутки данной темы

In [101]:
def descirbe_topic(topic_id, top_words=10, top_jokes=10, need_print=True, writing_object=None):
    most_relevant_words = ', '.join(x[0] for x in lda.show_topic(topic_id, topn=top_words))
    sample_jokes = '\n'.join(jokes_frame[jokes_frame.topic_id == topic_id].sample(top_jokes).joke_text.values)
    template = '=====\nTopic {}\nTop words: {}\nSample jokes:\n {}\n=====\n\n'.format(topic_id, most_relevant_words, sample_jokes)
    if need_print:
        print(template)
    if writing_object:
        writing_object.write(template)

In [102]:
with open('lda_100_topics_description.txt', 'w', encoding='utf-8') as f:
    for i in range(number_topics):
        descirbe_topic(topic_id=i, need_print=False, writing_object=f)

In [103]:
lda.save(fname='lda_100_topics.model')